In [ ]:
import pandas as pd

import numpy as np
from sklearn.tree import DecisionTreeClassifier
from typing import List, Tuple
import ast

In [34]:
test

,index,measure_duration_x,measure_duration_y,time_sig,x_tuples,y_tuples
0,195,6.778853,0.348837,4/4,"[(0.0, 0.050747812500000045), (0.4874460937500...","[(0.0, 0.1239583333333333), (0.5, 0.6239583333..."
1,196,6.778853,0.348837,4/4,"[(0.0, 0.04273499999999997), (0.24706171874999...","[(0.0, 0.12395833333333339), (0.75, 0.87395833..."
2,197,6.778853,0.348837,4/4,"[(0.0, 0.03872859374999926), (0.24439078124999...","[(0.0, 0.12395833333333339), (0.75, 0.99895833..."
3,198,6.778853,0.348837,4/4,"[(0.0, 0.23637796875000117), (0.21634593750000...","[(0.0, 0.2489583333333334), (0.25, 0.498958333..."
4,199,6.778853,0.348837,4/4,"[(0.0, 0.21100406249999892), (0.23504249999999...","[(0.0, 0.2489583333333325), (0.0, 0.2489583333..."
...,...,...,...,...,...,...
5341,18961,6.778853,0.348837,4/4,"[(0.0, 0.13621781249997866), (0.08547000000010...","[(0.0, 0.17559518541662555), (0.08928568749996..."
5342,18962,6.778853,0.348837,4/4,"[(0.0, 0.0761217187500165), (0.121527656250009...","[(0.0, 0.1755951854166824), (0.089285687500023..."
5343,18963,6.778853,0.348837,4/4,"[(0.0, 0.12954046875000813), (0.00133546875002...","[(0.0, 0.1755951854166824), (0.178571374999989..."
5344,18964,6.778853,0.348837,4/4,"[(0.0, 0.03739312499999414), (0.0, 0.052083281...","[(0.0, 0.17559518541662555), (0.17857137499998..."


In [ ]:
def process_data(df):
    df["x"] = df["x"].apply(ast.literal_eval)
    df["y"] = df["y"].apply(ast.literal_eval)
    df.drop(["perf_csv"], inplace=True, axis=1)
    df["measure_duration_x"] = df["x"][0][0]["measure_duration"]
    df["measure_duration_y"] = df["y"][0][0]["measure_duration"]
    df["time_sig"] = df["x"].apply(lambda x: x[0]["time_signature"])
    df_4_4 = df[df["time_sig"] == "4/4"]
    df_4_4["x_tuples"] = df_4_4["x"].apply(
        lambda lst: [(d["onset"], d["offset"]) for d in lst]
    )
    df_4_4["y_tuples"] = df_4_4["y"].apply(
        lambda lst: [(d["onset"], d["offset"]) for d in lst]
    )
    df_4_4.reset_index(inplace=True)
    df_4_4.drop(["measure", "x", "y"], axis=1, inplace=True)
    return df_4_4

In [ ]:
train = pd.read_csv("train.csv")
train = process_data(train)

validate = pd.read_csv("val.csv")
validate = process_data(validate)

test = pd.read_csv("test.csv")
test = process_data(test)

In [ ]:
round(test.measure_duration_y)

In [ ]:
def adjust_x_tuples(df):
    def adjust_tuples(tuples):
        offset = tuples[0][0]  # Get the 0th element of the 0th tuple
        return [(onset - offset, offset_ - offset) for onset, offset_ in tuples]

    df["y_tuples"] = df["y_tuples"].apply(
        lambda x: adjust_tuples(eval(x)) if isinstance(x, str) else adjust_tuples(x)
    )
    return df


train = adjust_x_tuples(train)
validate = adjust_x_tuples(validate)
test = adjust_x_tuples(test)

In [35]:
train

,index,measure_duration_x,measure_duration_y,time_sig,x_tuples,y_tuples
0,275,1.442058,1.5,4/4,"[(0.0, 0.36191203125000015), (0.92948625000000...","[(0.0, 1.2473958333333333), (1.25, 2.184895833..."
1,276,1.442058,1.5,4/4,"[(0.0, 1.3194431250000003), (1.113780937500000...","[(0.0, 1.247395833333333), (1.25, 2.4973958333..."
2,277,1.442058,1.5,4/4,"[(0.0, 0.42601453125000077), (0.00667734375000...","[(0.0, 1.2473958333333321), (0.0, 1.2473958333..."
3,278,1.442058,1.5,4/4,"[(0.0, 0.49545890625000055), (0.00534187499999...","[(0.0, 1.2473958333333321), (0.0, 1.2473958333..."
4,279,1.442058,1.5,4/4,"[(0.0, 0.06276703124999727), (0.04674140624999...","[(0.0, 0.07714860208333363), (0.07812516666666..."
...,...,...,...,...,...,...
45359,156275,1.442058,1.5,4/4,"[(0.0, 0.10950843749999706), (0.06810890625000...","[(0.0, 0.08451702291667118), (0.08522725000000..."
45360,156276,1.442058,1.5,4/4,"[(0.0, 0.12419859374999476), (0.10016015624999...","[(0.0, 0.08451702291667118), (0.08522725000000..."
45361,156277,1.442058,1.5,4/4,"[(0.0, 0.07211531250000291), (0.09615375000001...","[(0.0, 0.08451702291667118), (0.08522725000000..."
45362,156278,1.442058,1.5,4/4,"[(0.0, 0.0467414062500211), (0.400640625000022...","[(0.0, 0.2947442395833235), (0.045454533333327..."


In [ ]:
validate.head

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score


def manual_decision_tree_classifier(time):
    """
    Manually classify a time value to the nearest sixteenth note time using a decision tree approach

    Args:
        time: Float time value between 0 and 2

    Returns:
        Closest sixteenth note time
    """
    # Define all sixteenth note times
    sixteenth_note_times = [
        0.000,
        0.125,
        0.250,
        0.375,
        0.500,
        0.625,
        0.750,
        0.875,
        1.000,
        1.125,
        1.250,
        1.375,
        1.500,
        1.625,
        1.750,
        1.875,
        2.000,
    ]

    # First level: First half or second half of measure
    if time < 1.0:
        # Second level: First quarter or second quarter
        if time < 0.5:
            # Third level: First eighth or second eighth
            if time < 0.25:
                # Fourth level: First sixteenth or second sixteenth
                if time < 0.125:
                    return sixteenth_note_times[0]  # 0.000
                else:
                    return sixteenth_note_times[1]  # 0.125
            else:
                # Fourth level: Third sixteenth or fourth sixteenth
                if time < 0.375:
                    return sixteenth_note_times[2]  # 0.250
                else:
                    return sixteenth_note_times[3]  # 0.375
        else:
            # Third level: Third eighth or fourth eighth
            if time < 0.75:
                # Fourth level: Fifth sixteenth or sixth sixteenth
                if time < 0.625:
                    return sixteenth_note_times[4]  # 0.500
                else:
                    return sixteenth_note_times[5]  # 0.625
            else:
                # Fourth level: Seventh sixteenth or eighth sixteenth
                if time < 0.875:
                    return sixteenth_note_times[6]  # 0.750
                else:
                    return sixteenth_note_times[7]  # 0.875
    else:
        # Second level: Third quarter or fourth quarter
        if time < 1.5:
            # Third level: Fifth eighth or sixth eighth
            if time < 1.25:
                # Fourth level: Ninth sixteenth or tenth sixteenth
                if time < 1.125:
                    return sixteenth_note_times[8]  # 1.000
                else:
                    return sixteenth_note_times[9]  # 1.125
            else:
                # Fourth level: Eleventh sixteenth or twelfth sixteenth
                if time < 1.375:
                    return sixteenth_note_times[10]  # 1.250
                else:
                    return sixteenth_note_times[11]  # 1.375
        else:
            # Third level: Seventh eighth or eighth eighth
            if time < 1.75:
                # Fourth level: Thirteenth sixteenth or fourteenth sixteenth
                if time < 1.625:
                    return sixteenth_note_times[12]  # 1.500
                else:
                    return sixteenth_note_times[13]  # 1.625
            else:
                # Fourth level: Fifteenth sixteenth or sixteenth sixteenth
                if time < 1.875:
                    return sixteenth_note_times[14]  # 1.750
                else:
                    return (
                        sixteenth_note_times[15]
                        if time < 2.0
                        else sixteenth_note_times[16]
                    )  # 1.875 or 2.000


def normalize_midi_times(tuples_list, current_duration, target_duration=2.0):
    """
    Normalize MIDI times to fit within a target duration (typically 2 seconds)

    Args:
        tuples_list: List of (onset, offset) tuples
        current_duration: Current duration of the measure
        target_duration: Target duration to normalize to (default: 2.0)

    Returns:
        List of normalized (onset, offset) tuples
    """
    scaling_factor = target_duration / current_duration
    normalized_tuples = [
        (onset * scaling_factor, offset * scaling_factor)
        for onset, offset in tuples_list
    ]
    return normalized_tuples


def classify_tuples(normalized_tuples):
    """
    Classify each time in normalized tuples to nearest sixteenth note time

    Args:
        normalized_tuples: List of normalized (onset, offset) tuples

    Returns:
        List of classified (onset, offset) tuples
    """
    classified_tuples = []
    for onset, offset in normalized_tuples:
        classified_onset = manual_decision_tree_classifier(onset)
        classified_offset = manual_decision_tree_classifier(offset)
        classified_tuples.append((classified_onset, classified_offset))
    return classified_tuples


def denormalize_midi_times(classified_tuples, target_duration):
    """
    Stretch or compress classified tuples to match target duration

    Args:
        classified_tuples: List of classified (onset, offset) tuples
        target_duration: Target duration to stretch/compress to

    Returns:
        List of denormalized (onset, offset) tuples
    """
    scaling_factor = (
        target_duration / 2.0
    )  # Assuming classification was done for 2.0 seconds
    denormalized_tuples = [
        (onset * scaling_factor, offset * scaling_factor)
        for onset, offset in classified_tuples
    ]
    return denormalized_tuples


def process_midi_data(df):
    """
    Process MIDI data: normalize performance MIDI, classify to sixteenth notes,
    then stretch to match pure MIDI duration

    Args:
        df: DataFrame with 'x_tuples', 'measure_duration_x', 'measure_duration_y' columns

    Returns:
        DataFrame with predicted column added
    """
    df = df.copy()
    predictions = []

    for i, row in df.iterrows():
        # Extract performance MIDI tuples and durations
        performance_tuples = (
            eval(row["x_tuples"])
            if isinstance(row["x_tuples"], str)
            else row["x_tuples"]
        )
        performance_duration = row["measure_duration_x"]
        pure_midi_duration = row["measure_duration_y"]

        # Step 1: Normalize performance MIDI to 2 seconds
        normalized_tuples = normalize_midi_times(
            performance_tuples, performance_duration
        )

        # Step 2: Classify to sixteenth notes
        classified_tuples = classify_tuples(normalized_tuples)

        # Step 3: Stretch classified tuples to match pure MIDI duration
        final_tuples = denormalize_midi_times(classified_tuples, pure_midi_duration)

        predictions.append(final_tuples)

    df["predicted"] = predictions
    print(df)
    return df


def evaluate_classifier(y_true, y_pred):
    """
    Evaluate the classifier by comparing predicted tuples with ground truth

    Args:
        y_true: List of ground truth tuples
        y_pred: List of predicted tuples

    Returns:
        Accuracy score
    """
    # Convert to string representation for comparison
    y_true_str = [str(tup) for tup in y_true]
    y_pred_str = [str(tup) for tup in y_pred]

    # Calculate accuracy
    accuracy = sum(
        1 for true, pred in zip(y_true_str, y_pred_str) if true == pred
    ) / len(y_true)
    return accuracy


def main(train_df, test_df, valid_df):

    print(
        f"Loaded datasets - Train: {len(train_df)} rows, Test: {len(test_df)} rows, Validation: {len(valid_df)} rows"
    )

    # Process training data
    print("Processing training data...")
    train_df = process_midi_data(train_df)

    # Process test data
    print("Processing test data...")
    test_df = process_midi_data(test_df)

    # Process validation data
    print("Processing validation data...")
    valid_df = process_midi_data(valid_df)

    # Evaluate on test set
    print("Evaluating on test set...")
    y_true = [eval(y) if isinstance(y, str) else y for y in test_df["y_tuples"]]
    y_pred = test_df["predicted"]

    test_accuracy = evaluate_classifier(y_true, y_pred)
    print(f"Test accuracy: {test_accuracy:.4f}")

    # Also evaluate on validation set
    print("Evaluating on validation set...")
    y_true_val = [eval(y) if isinstance(y, str) else y for y in valid_df["y_tuples"]]
    y_pred_val = valid_df["predicted"]

    val_accuracy = evaluate_classifier(y_true_val, y_pred_val)
    print(f"Validation accuracy: {val_accuracy:.4f}")

In [ ]:
main(train, test, validate)

In [ ]:
main(test, train, validate)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Define the sixteenth note times we'll classify to
sixteenth_note_times = [
    0.000,
    0.125,
    0.250,
    0.375,
    0.500,
    0.625,
    0.750,
    0.875,
    1.000,
    1.125,
    1.250,
    1.375,
    1.500,
    1.625,
    1.750,
    1.875,
    2.000,
]


def normalize_time_to_2sec(time_value, measure_duration):
    """Normalize a time value to fit within a 2-second measure"""
    return (time_value / measure_duration) * 2.0


def denormalize_time(time_value, target_duration):
    """Scale a normalized time value to a target duration"""
    return (time_value / 2.0) * target_duration


def find_closest_sixteenth_note(time_value):
    """Find the closest sixteenth note time to the given time value"""
    return sixteenth_note_times[
        np.argmin(np.abs(np.array(sixteenth_note_times) - time_value))
    ]


def process_note_tuples(tuples_list, original_duration, target_duration=None):
    """
    Process a list of (onset, offset) tuples:
    1. Normalize to 2-second duration
    2. Find closest sixteenth note times
    3. Optionally scale to target duration
    """
    normalized_tuples = []
    classified_tuples = []
    result_tuples = []

    # Normalize to 2-second duration
    for onset, offset in tuples_list:
        norm_onset = normalize_time_to_2sec(onset, original_duration)
        norm_offset = normalize_time_to_2sec(offset, original_duration)
        normalized_tuples.append((norm_onset, norm_offset))

        # Classify to closest sixteenth note times
        class_onset = find_closest_sixteenth_note(norm_onset)
        class_offset = find_closest_sixteenth_note(norm_offset)
        classified_tuples.append((class_onset, class_offset))

        # If target duration provided, scale back
        if target_duration is not None:
            final_onset = denormalize_time(class_onset, target_duration)
            final_offset = denormalize_time(class_offset, target_duration)
            result_tuples.append((final_onset, final_offset))

    if target_duration is not None:
        return result_tuples
    else:
        return classified_tuples


def prepare_training_data(df):
    """
    Prepare training data for the decision tree classifier
    Returns:
    - X: Features (normalized times)
    - y: Target values (index of closest sixteenth note)
    """
    X = []  # Features will be normalized times
    y = []  # Targets will be index of sixteenth note time

    for _, row in df.iterrows():
        performance_tuples = (
            eval(row["x_tuples"])
            if isinstance(row["x_tuples"], str)
            else row["x_tuples"]
        )
        performance_duration = row["measure_duration_x"]
        pure_tuples = (
            eval(row["y_tuples"])
            if isinstance(row["y_tuples"], str)
            else row["y_tuples"]
        )

        # Process each note in the performance tuples
        for onset, offset in performance_tuples:
            # Normalize to 2 seconds
            norm_onset = normalize_time_to_2sec(onset, performance_duration)
            norm_offset = normalize_time_to_2sec(offset, performance_duration)

            # Find closest sixteenth note indices
            onset_idx = np.argmin(np.abs(np.array(sixteenth_note_times) - norm_onset))
            offset_idx = np.argmin(np.abs(np.array(sixteenth_note_times) - norm_offset))

            # Add to training data
            X.append([norm_onset])
            y.append(onset_idx)

            X.append([norm_offset])
            y.append(offset_idx)

    return np.array(X), np.array(y)


def train_decision_tree(X, y, max_depth=5):
    """Train a decision tree classifier"""
    dt_classifier = DecisionTreeClassifier(max_depth=max_depth)
    dt_classifier.fit(X, y)
    return dt_classifier


def classify_with_decision_tree(model, df):
    """
    Classify note times using the trained decision tree model
    Returns a new dataframe with predicted y_tuples
    """
    df_result = df.copy()
    df_result["y_pred_tuples"] = None

    for idx, row in df.iterrows():
        performance_tuples = (
            eval(row["x_tuples"])
            if isinstance(row["x_tuples"], str)
            else row["x_tuples"]
        )
        performance_duration = row["measure_duration_x"]
        pure_duration = row["measure_duration_y"]

        predicted_tuples = []

        for onset, offset in performance_tuples:
            # Normalize to 2 seconds
            norm_onset = normalize_time_to_2sec(onset, performance_duration)
            norm_offset = normalize_time_to_2sec(offset, performance_duration)

            # Predict sixteenth note indices
            onset_idx = model.predict([[norm_onset]])[0]
            offset_idx = model.predict([[norm_offset]])[0]

            # Get the actual sixteenth note times
            class_onset = sixteenth_note_times[onset_idx]
            class_offset = sixteenth_note_times[offset_idx]

            # Scale to pure midi duration
            final_onset = denormalize_time(class_onset, pure_duration)
            final_offset = denormalize_time(class_offset, pure_duration)

            predicted_tuples.append((final_onset, final_offset))

        df_result.at[idx, "y_pred_tuples"] = predicted_tuples

    return df_result


def evaluate_model(df_test, df_pred):
    """
    Evaluate the model predictions against the ground truth
    Returns accuracy metrics
    """
    total_notes = 0
    correct_onsets = 0
    correct_offsets = 0

    for idx, row in df_test.iterrows():
        true_tuples = (
            eval(row["y_tuples"])
            if isinstance(row["y_tuples"], str)
            else row["y_tuples"]
        )
        pred_tuples = df_pred.loc[idx, "y_pred_tuples"]

        # Count only if we have the same number of notes
        if len(true_tuples) == len(pred_tuples):
            # Sort both by onset time for fair comparison
            true_tuples = sorted(true_tuples, key=lambda x: x[0])
            pred_tuples = sorted(pred_tuples, key=lambda x: x[0])

            for (true_onset, true_offset), (pred_onset, pred_offset) in zip(
                true_tuples, pred_tuples
            ):
                total_notes += 1

                # Use a small tolerance for comparison
                if abs(true_onset - pred_onset) < 0.01:
                    correct_onsets += 1
                if abs(true_offset - pred_offset) < 0.01:
                    correct_offsets += 1

    onset_accuracy = correct_onsets / total_notes if total_notes > 0 else 0
    offset_accuracy = correct_offsets / total_notes if total_notes > 0 else 0

    return {
        "onset_accuracy": onset_accuracy,
        "offset_accuracy": offset_accuracy,
        "total_notes": total_notes,
    }


# Main execution flow
def main(train_df, test_df, val_df):
    """Main function to train, predict and evaluate the model"""
    # Handle string to list conversion if needed
    for df in [train_df, test_df, val_df]:
        if isinstance(df["x_tuples"].iloc[0], str):
            df["x_tuples"] = df["x_tuples"].apply(eval)
        if isinstance(df["y_tuples"].iloc[0], str):
            df["y_tuples"] = df["y_tuples"].apply(eval)

    # Prepare training data
    X_train, y_train = prepare_training_data(train_df)

    # Try different max_depth values
    best_depth = 5
    best_val_accuracy = 0

    for depth in range(3, 11):
        # Train model
        model = train_decision_tree(X_train, y_train, max_depth=depth)

        # Predict on validation set
        val_pred_df = classify_with_decision_tree(model, val_df)

        # Evaluate
        val_metrics = evaluate_model(val_df, val_pred_df)
        mean_accuracy = (
            val_metrics["onset_accuracy"] + val_metrics["offset_accuracy"]
        ) / 2

        print(f"Max depth {depth}: Validation accuracy = {mean_accuracy:.4f}")

        if mean_accuracy > best_val_accuracy:
            best_val_accuracy = mean_accuracy
            best_depth = depth

    print(
        f"Best max_depth: {best_depth} with validation accuracy: {best_val_accuracy:.4f}"
    )

    # Train final model with best depth
    final_model = train_decision_tree(X_train, y_train, max_depth=best_depth)

    # Predict on test set
    test_pred_df = classify_with_decision_tree(final_model, test_df)

    # Evaluate on test set
    test_metrics = evaluate_model(test_df, test_pred_df)
    print(f"Test onset accuracy: {test_metrics['onset_accuracy']:.4f}")
    print(f"Test offset accuracy: {test_metrics['offset_accuracy']:.4f}")

    return final_model, test_pred_df


# Example usage:
model, predictions = main(train, test, validate)

In [ ]:
model, predictions = main(train, validate, test)

In [ ]:
model, predictions = main(test, train, validate)

In [ ]:
model, predictions = main(test, validate, train)

In [ ]:
model, predictions = main(validate, train, test)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


def visualize_test_accuracy(test_df, test_pred_df):
    """
    Clearly visualize the test accuracy metrics
    """
    # Calculate test metrics
    metrics = evaluate_manual_model(test_df, test_pred_df)

    # Create a bar chart for the accuracy metrics
    labels = ["Onset Accuracy", "Offset Accuracy"]
    values = [metrics["onset_accuracy"], metrics["offset_accuracy"]]

    fig, ax = plt.subplots(figsize=(10, 6))
    bars = ax.bar(labels, values, color=["#3498db", "#2ecc71"])

    # Add value labels on top of bars
    for bar in bars:
        height = bar.get_height()
        ax.text(
            bar.get_x() + bar.get_width() / 2,
            height + 0.01,
            f"{height:.4f}",
            ha="center",
            fontweight="bold",
        )

    # Add styling
    ax.set_ylim(0, 1.0)
    ax.set_ylabel("Accuracy Score", fontsize=12)
    ax.set_title("Manual Decision Tree Test Accuracy", fontsize=16, fontweight="bold")
    ax.grid(axis="y", linestyle="--", alpha=0.7)

    # Add total notes count as text
    plt.text(
        0.5,
        0.02,
        f"Total notes evaluated: {metrics['total_notes']}",
        ha="center",
        transform=fig.transFigure,
    )

    plt.tight_layout()
    plt.savefig("manual_tree_test_accuracy.png", dpi=300)
    plt.show()

    print(f"\n===== TEST ACCURACY SUMMARY =====")
    print(f"Onset Accuracy:  {metrics['onset_accuracy']:.4f}")
    print(f"Offset Accuracy: {metrics['offset_accuracy']:.4f}")
    print(f"Total Notes:     {metrics['total_notes']}")
    print(f"================================")

    return metrics


# Helper function from original code
def evaluate_manual_model(df_test, df_pred):
    """
    Evaluate the manual model predictions against the ground truth
    Returns accuracy metrics
    """
    total_notes = 0
    correct_onsets = 0
    correct_offsets = 0

    for idx, row in df_test.iterrows():
        true_tuples = (
            eval(row["y_tuples"])
            if isinstance(row["y_tuples"], str)
            else row["y_tuples"]
        )
        pred_tuples = df_pred.loc[idx, "y_pred_tuples"]

        # Count only if we have the same number of notes
        if len(true_tuples) == len(pred_tuples):
            # Sort both by onset time for fair comparison
            true_tuples = sorted(true_tuples, key=lambda x: x[0])
            pred_tuples = sorted(pred_tuples, key=lambda x: x[0])

            for (true_onset, true_offset), (pred_onset, pred_offset) in zip(
                true_tuples, pred_tuples
            ):
                total_notes += 1

                # Use a small tolerance for comparison
                if abs(true_onset - pred_onset) < 0.01:
                    correct_onsets += 1
                if abs(true_offset - pred_offset) < 0.01:
                    correct_offsets += 1

    onset_accuracy = correct_onsets / total_notes if total_notes > 0 else 0
    offset_accuracy = correct_offsets / total_notes if total_notes > 0 else 0

    return {
        "onset_accuracy": onset_accuracy,
        "offset_accuracy": offset_accuracy,
        "total_notes": total_notes,
    }


# Example usage:
# Assuming manual_tree, test_df, and test_pred_df are defined:
# metrics = visualize_test_accuracy(test_df, test_pred_df)

# To use this with the existing code:
# 1. Run your code to get the manual_tree and predictions
# 2. Then call this function with:
#    metrics = visualize_test_accuracy(test, predictions)

In [ ]:
import numpy as np
import pandas as pd
from typing import List, Tuple, Union

# Define the standard sixteenth note times in a 2-second measure
SIXTEENTH_NOTE_TIMES = [
    0.000,
    0.125,
    0.250,
    0.375,
    0.500,
    0.625,
    0.750,
    0.875,
    1.000,
    1.125,
    1.250,
    1.375,
    1.500,
    1.625,
    1.750,
    1.875,
    2.000,
]


def decision_tree_classifier(time: float) -> float:
    """
    Manual decision tree classifier to map a time value to the nearest sixteenth note time.
    This uses a hierarchical approach:
    1. First determines if time is in first half (0-1s) or second half (1-2s) of measure
    2. Then determines which quarter of the measure
    3. Finally determines which sixteenth note within that quarter

    Args:
        time: Normalized time value between 0 and 2

    Returns:
        The nearest sixteenth note time
    """
    # Ensure time is in valid range
    time = max(0.0, min(2.0, time))

    # First level: Is it in first half (0-1s) or second half (1-2s)?
    if time < 1.0:
        # Second level: First quarter (0-0.5s) or second quarter (0.5-1s)?
        if time < 0.5:
            # Third level: Which sixteenth note in the first quarter?
            if time < 0.25:
                # Fourth level: First or second sixteenth?
                return 0.0 if time < 0.125 else 0.125
            else:
                # Fourth level: Third or fourth sixteenth?
                return 0.25 if time < 0.375 else 0.375
        else:
            # Third level: Which sixteenth note in the second quarter?
            if time < 0.75:
                # Fourth level: Fifth or sixth sixteenth?
                return 0.5 if time < 0.625 else 0.625
            else:
                # Fourth level: Seventh or eighth sixteenth?
                return 0.75 if time < 0.875 else 0.875
    else:
        # Second level: Third quarter (1-1.5s) or fourth quarter (1.5-2s)?
        if time < 1.5:
            # Third level: Which sixteenth note in the third quarter?
            if time < 1.25:
                # Fourth level: Ninth or tenth sixteenth?
                return 1.0 if time < 1.125 else 1.125
            else:
                # Fourth level: Eleventh or twelfth sixteenth?
                return 1.25 if time < 1.375 else 1.375
        else:
            # Third level: Which sixteenth note in the fourth quarter?
            if time < 1.75:
                # Fourth level: Thirteenth or fourteenth sixteenth?
                return 1.5 if time < 1.625 else 1.625
            else:
                # Fourth level: Fifteenth or sixteenth sixteenth?
                return 1.75 if time < 1.875 else 2.0


def normalize_times(
    tuples_list: List[Tuple[float, float]],
    original_duration: float,
    target_duration: float = 2.0,
) -> List[Tuple[float, float]]:
    """
    Normalize all time values in a list of tuples to fit within target_duration.

    Args:
        tuples_list: List of (start_time, end_time) tuples
        original_duration: Original duration of the measure
        target_duration: Target duration (default 2.0 seconds)

    Returns:
        List of normalized (start_time, end_time) tuples
    """
    scale_factor = target_duration / original_duration
    return [(start * scale_factor, end * scale_factor) for start, end in tuples_list]


def denormalize_times(
    tuples_list: List[Tuple[float, float]],
    target_duration: float,
    source_duration: float = 2.0,
) -> List[Tuple[float, float]]:
    """
    Scale classified time values to match target duration.

    Args:
        tuples_list: List of (start_time, end_time) tuples
        target_duration: Target duration for the measure
        source_duration: Source duration (default 2.0 seconds)

    Returns:
        List of denormalized (start_time, end_time) tuples
    """
    scale_factor = target_duration / source_duration
    return [(start * scale_factor, end * scale_factor) for start, end in tuples_list]


def classify_note_times(
    tuples_list: List[Tuple[float, float]],
) -> List[Tuple[float, float]]:
    """
    Classify each time in a list of tuples to the nearest sixteenth note time.

    Args:
        tuples_list: List of normalized (start_time, end_time) tuples

    Returns:
        List of classified (start_time, end_time) tuples
    """
    return [
        (decision_tree_classifier(start), decision_tree_classifier(end))
        for start, end in tuples_list
    ]


def process_measure(row: pd.Series) -> List[Tuple[float, float]]:
    """
    Process a single measure row:
    1. Extract the performance MIDI tuples
    2. Normalize to 2-second duration
    3. Classify to sixteenth notes
    4. Rescale to match pure MIDI duration

    Args:
        row: DataFrame row containing measure data

    Returns:
        Processed tuples list matching pure MIDI duration
    """
    # Extract values from the row
    performance_tuples = (
        eval(row["x_tuples"]) if isinstance(row["x_tuples"], str) else row["x_tuples"]
    )
    performance_duration = row["measure_duration_x"]
    pure_midi_duration = row["measure_duration_y"]

    # Step 1: Normalize to 2-second duration
    normalized_tuples = normalize_times(performance_tuples, performance_duration)

    # Step 2: Classify to sixteenth notes
    classified_tuples = classify_note_times(normalized_tuples)

    # Step 3: Rescale to match pure MIDI duration
    rescaled_tuples = denormalize_times(classified_tuples, pure_midi_duration)

    return rescaled_tuples


def process_dataset(df: pd.DataFrame) -> pd.DataFrame:
    """
    Process the entire dataset.

    Args:
        df: DataFrame containing measures data

    Returns:
        DataFrame with added predicted column
    """
    # Create a copy to avoid modifying the original
    result_df = df.copy()

    # Apply processing to each row
    result_df["predicted"] = result_df.apply(process_measure, axis=1)

    return result_df

In [ ]:
manual_tree, predictions, analysis = manual_decision_tree_main(train, test, validate)
metrics = visualize_test_accuracy(test, predictions)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import seaborn as sns

# Define the sixteenth note times we'll classify to
sixteenth_note_times = [
    0.000,
    0.125,
    0.250,
    0.375,
    0.500,
    0.625,
    0.750,
    0.875,
    1.000,
    1.125,
    1.250,
    1.375,
    1.500,
    1.625,
    1.750,
    1.875,
    2.000,
]


def normalize_time_to_2sec(time_value, measure_duration):
    """Normalize a time value to fit within a 2-second measure"""
    return (time_value / measure_duration) * 2.0


def denormalize_time(time_value, target_duration):
    """Scale a normalized time value to a target duration"""
    return (time_value / 2.0) * target_duration


class ManualDecisionTree:
    """
    A manually defined hierarchical decision tree for classifying note times
    into sixteenth note positions.
    """

    def __init__(self):
        # Store the sixteenth note times
        self.sixteenth_note_times = sixteenth_note_times

        # Define the hierarchical boundaries for our decision tree
        # Level 1: Half-measure boundaries (0-1, 1-2)
        self.half_boundary = 1.0

        # Level 2: Quarter-measure boundaries
        self.quarter_boundaries = [0.5, 1.5]

        # Level 3: Eighth-note boundaries
        self.eighth_boundaries = [0.25, 0.75, 1.25, 1.75]

        # Level 4: Sixteenth-note boundaries
        self.sixteenth_boundaries = [
            0.125,
            0.375,
            0.625,
            0.875,
            1.125,
            1.375,
            1.625,
            1.875,
        ]

    def classify(self, time_value):
        """
        Classify a normalized time value (0-2 seconds) using our manual
        hierarchical decision tree approach.

        Returns the closest sixteenth note time.
        """
        # First decision: first half or second half of measure
        if time_value <= self.half_boundary:
            # In first half of measure

            # Second decision: first quarter or second quarter
            if time_value <= self.quarter_boundaries[0]:
                # In first quarter of measure

                # Third decision: first eighth or second eighth
                if time_value <= self.eighth_boundaries[0]:
                    # In first eighth of measure

                    # Fourth decision: first sixteenth or second sixteenth
                    if time_value <= self.sixteenth_boundaries[0]:
                        return 0.000  # First sixteenth note
                    else:
                        return 0.125  # Second sixteenth note

                else:
                    # In second eighth of measure

                    # Fourth decision: third sixteenth or fourth sixteenth
                    if time_value <= self.sixteenth_boundaries[1]:
                        return 0.250  # Third sixteenth note
                    else:
                        return 0.375  # Fourth sixteenth note

            else:
                # In second quarter of measure

                # Third decision: third eighth or fourth eighth
                if time_value <= self.eighth_boundaries[1]:
                    # In third eighth of measure

                    # Fourth decision
                    if time_value <= self.sixteenth_boundaries[2]:
                        return 0.500  # Fifth sixteenth note
                    else:
                        return 0.625  # Sixth sixteenth note

                else:
                    # In fourth eighth of measure

                    # Fourth decision
                    if time_value <= self.sixteenth_boundaries[3]:
                        return 0.750  # Seventh sixteenth note
                    else:
                        return 0.875  # Eighth sixteenth note

        else:
            # In second half of measure

            # Second decision: third quarter or fourth quarter
            if time_value <= self.quarter_boundaries[1]:
                # In third quarter of measure

                # Third decision: fifth eighth or sixth eighth
                if time_value <= self.eighth_boundaries[2]:
                    # In fifth eighth of measure

                    # Fourth decision
                    if time_value <= self.sixteenth_boundaries[4]:
                        return 1.000  # Ninth sixteenth note
                    else:
                        return 1.125  # Tenth sixteenth note

                else:
                    # In sixth eighth of measure

                    # Fourth decision
                    if time_value <= self.sixteenth_boundaries[5]:
                        return 1.250  # Eleventh sixteenth note
                    else:
                        return 1.375  # Twelfth sixteenth note

            else:
                # In fourth quarter of measure

                # Third decision: seventh eighth or eighth eighth
                if time_value <= self.eighth_boundaries[3]:
                    # In seventh eighth of measure

                    # Fourth decision
                    if time_value <= self.sixteenth_boundaries[6]:
                        return 1.500  # Thirteenth sixteenth note
                    else:
                        return 1.625  # Fourteenth sixteenth note

                else:
                    # In eighth eighth of measure

                    # Fourth decision
                    if time_value <= self.sixteenth_boundaries[7]:
                        return 1.750  # Fifteenth sixteenth note
                    else:
                        return 2.000  # Sixteenth sixteenth note (end of measure)

    def visualize_tree(self):
        """Generate a visualization of the manual decision tree's decision boundaries"""
        # Create a range of times to test (0 to 2 seconds with fine granularity)
        times = np.linspace(0, 2, 1000)

        # Predict the category for each time
        predictions = [self.classify(t) for t in times]

        # Plot the mapping from input time to nearest sixteenth note
        plt.figure(figsize=(15, 8))
        plt.plot(times, predictions, "b-", linewidth=2)

        # Mark the actual sixteenth note positions
        for time in sixteenth_note_times:
            plt.axhline(y=time, color="r", linestyle="--", alpha=0.3)

        # Add decision boundaries
        all_boundaries = (
            [self.half_boundary]
            + self.quarter_boundaries
            + self.eighth_boundaries
            + self.sixteenth_boundaries
        )

        for boundary in sorted(all_boundaries):
            plt.axvline(x=boundary, color="g", linestyle="-", alpha=0.7)

        # Add labels for decision levels
        boundary_labels = {
            self.half_boundary: "Half Measure",
            self.quarter_boundaries[0]: "Quarter 1/2",
            self.quarter_boundaries[1]: "Quarter 3/4",
            self.eighth_boundaries[0]: "Eighth 1/2",
            self.eighth_boundaries[1]: "Eighth 3/4",
            self.eighth_boundaries[2]: "Eighth 5/6",
            self.eighth_boundaries[3]: "Eighth 7/8",
        }

        for boundary, label in boundary_labels.items():
            plt.text(
                boundary,
                2.05,
                label,
                rotation=90,
                verticalalignment="bottom",
                horizontalalignment="center",
            )

        plt.xlabel("Input Time (seconds)")
        plt.ylabel("Classified Time (seconds)")
        plt.title("Manual Hierarchical Decision Tree Classification Boundaries")
        plt.grid(True)
        plt.ylim(-0.1, 2.1)
        plt.savefig("manual_decision_boundaries.png", dpi=300)
        plt.show()

    def print_decision_structure(self):
        """Print out the decision structure in a readable format"""
        print("Manual Decision Tree Structure:")
        print("Level 1: Is time <= 1.0?")
        print("  If YES (First half of measure):")
        print("    Level 2: Is time <= 0.5?")
        print("      If YES (First quarter):")
        print("        Level 3: Is time <= 0.25?")
        print("          If YES (First eighth): Is time <= 0.125? → 0.0 : 0.125")
        print("          If NO (Second eighth): Is time <= 0.375? → 0.25 : 0.375")
        print("      If NO (Second quarter):")
        print("        Level 3: Is time <= 0.75?")
        print("          If YES (Third eighth): Is time <= 0.625? → 0.5 : 0.625")
        print("          If NO (Fourth eighth): Is time <= 0.875? → 0.75 : 0.875")
        print("  If NO (Second half of measure):")
        print("    Level 2: Is time <= 1.5?")
        print("      If YES (Third quarter):")
        print("        Level 3: Is time <= 1.25?")
        print("          If YES (Fifth eighth): Is time <= 1.125? → 1.0 : 1.125")
        print("          If NO (Sixth eighth): Is time <= 1.375? → 1.25 : 1.375")
        print("      If NO (Fourth quarter):")
        print("        Level 3: Is time <= 1.75?")
        print("          If YES (Seventh eighth): Is time <= 1.625? → 1.5 : 1.625")
        print("          If NO (Eighth eighth): Is time <= 1.875? → 1.75 : 2.0")


def process_note_tuples_manual(
    tuples_list, original_duration, manual_tree, target_duration=None
):
    """
    Process a list of (onset, offset) tuples using the manual decision tree:
    1. Normalize to 2-second duration
    2. Classify using the manual decision tree
    3. Optionally scale to target duration
    """
    normalized_tuples = []
    classified_tuples = []
    result_tuples = []

    # Normalize to 2-second duration
    for onset, offset in tuples_list:
        norm_onset = normalize_time_to_2sec(onset, original_duration)
        norm_offset = normalize_time_to_2sec(offset, original_duration)
        normalized_tuples.append((norm_onset, norm_offset))

        # Classify using manual decision tree
        class_onset = manual_tree.classify(norm_onset)
        class_offset = manual_tree.classify(norm_offset)
        classified_tuples.append((class_onset, class_offset))

        # If target duration provided, scale back
        if target_duration is not None:
            final_onset = denormalize_time(class_onset, target_duration)
            final_offset = denormalize_time(class_offset, target_duration)
            result_tuples.append((final_onset, final_offset))

    if target_duration is not None:
        return result_tuples
    else:
        return classified_tuples


def classify_with_manual_tree(manual_tree, df):
    """
    Classify note times using the manual decision tree
    Returns a new dataframe with predicted y_tuples
    """
    df_result = df.copy()
    df_result["y_pred_tuples"] = None

    for idx, row in df.iterrows():
        performance_tuples = (
            eval(row["x_tuples"])
            if isinstance(row["x_tuples"], str)
            else row["x_tuples"]
        )
        performance_duration = row["measure_duration_x"]
        pure_duration = row["measure_duration_y"]

        predicted_tuples = process_note_tuples_manual(
            performance_tuples,
            performance_duration,
            manual_tree,
            target_duration=pure_duration,
        )

        df_result.at[idx, "y_pred_tuples"] = predicted_tuples

    return df_result


def evaluate_manual_model(df_test, df_pred):
    """
    Evaluate the manual model predictions against the ground truth
    Returns accuracy metrics
    """
    total_notes = 0
    correct_onsets = 0
    correct_offsets = 0

    for idx, row in df_test.iterrows():
        true_tuples = (
            eval(row["y_tuples"])
            if isinstance(row["y_tuples"], str)
            else row["y_tuples"]
        )
        pred_tuples = df_pred.loc[idx, "y_pred_tuples"]

        # Count only if we have the same number of notes
        if len(true_tuples) == len(pred_tuples):
            # Sort both by onset time for fair comparison
            true_tuples = sorted(true_tuples, key=lambda x: x[0])
            pred_tuples = sorted(pred_tuples, key=lambda x: x[0])

            for (true_onset, true_offset), (pred_onset, pred_offset) in zip(
                true_tuples, pred_tuples
            ):
                total_notes += 1

                # Use a small tolerance for comparison
                if abs(true_onset - pred_onset) < 0.01:
                    correct_onsets += 1
                if abs(true_offset - pred_offset) < 0.01:
                    correct_offsets += 1

    onset_accuracy = correct_onsets / total_notes if total_notes > 0 else 0
    offset_accuracy = correct_offsets / total_notes if total_notes > 0 else 0

    return {
        "onset_accuracy": onset_accuracy,
        "offset_accuracy": offset_accuracy,
        "total_notes": total_notes,
    }


def analyze_classification_results_manual(manual_tree, test_df):
    """Analyze how well the manual tree classifies times across the measure"""
    all_norm_times = []
    all_true_classifications = []
    all_predictions = []

    for _, row in test_df.iterrows():
        performance_tuples = (
            eval(row["x_tuples"])
            if isinstance(row["x_tuples"], str)
            else row["x_tuples"]
        )
        performance_duration = row["measure_duration_x"]

        for onset, offset in performance_tuples:
            # Process onset
            norm_onset = normalize_time_to_2sec(onset, performance_duration)
            # Find the ground truth classification (closest sixteenth note)
            true_onset_idx = np.argmin(
                np.abs(np.array(sixteenth_note_times) - norm_onset)
            )
            true_onset_time = sixteenth_note_times[true_onset_idx]
            # Get manual tree prediction
            pred_onset_time = manual_tree.classify(norm_onset)
            pred_onset_idx = sixteenth_note_times.index(pred_onset_time)

            all_norm_times.append(norm_onset)
            all_true_classifications.append(true_onset_idx)
            all_predictions.append(pred_onset_idx)

            # Process offset (same logic)
            norm_offset = normalize_time_to_2sec(offset, performance_duration)
            true_offset_idx = np.argmin(
                np.abs(np.array(sixteenth_note_times) - norm_offset)
            )
            true_offset_time = sixteenth_note_times[true_offset_idx]
            pred_offset_time = manual_tree.classify(norm_offset)
            pred_offset_idx = sixteenth_note_times.index(pred_offset_time)

            all_norm_times.append(norm_offset)
            all_true_classifications.append(true_offset_idx)
            all_predictions.append(pred_offset_idx)

    # Create a confusion matrix
    cm = np.zeros((len(sixteenth_note_times), len(sixteenth_note_times)), dtype=int)
    for true_idx, pred_idx in zip(all_true_classifications, all_predictions):
        cm[true_idx, pred_idx] += 1

    # Plot the confusion matrix
    # plt.figure(figsize=(12, 10))
    # sns.heatmap(
    #     cm,
    #     annot=True,
    #     fmt="d",
    #     cmap="Blues",
    #     xticklabels=[f"{t:.3f}" for t in sixteenth_note_times],
    #     yticklabels=[f"{t:.3f}" for t in sixteenth_note_times],
    # )
    # plt.xlabel("Predicted Class")

    # plt.ylabel("True Class")
    # plt.title("Confusion Matrix for Manual Decision Tree Classification")
    # plt.tight_layout()
    # plt.savefig("manual_confusion_matrix.png", dpi=300)
    # plt.show()

    # Analyze accuracy by position in measure
    accuracy_by_position = {}
    for true_idx in range(len(sixteenth_note_times)):
        indices = [i for i, c in enumerate(all_true_classifications) if c == true_idx]
        if indices:
            correct = sum(
                1 for i in indices if all_predictions[i] == all_true_classifications[i]
            )
            accuracy_by_position[sixteenth_note_times[true_idx]] = correct / len(
                indices
            )

    # Plot accuracy by position
    positions = list(accuracy_by_position.keys())
    accuracies = list(accuracy_by_position.values())

    # plt.figure(figsize=(12, 6))
    # plt.bar(positions, accuracies)
    # plt.xlabel("Position in Measure (seconds)")
    # plt.ylabel("Classification Accuracy")
    # plt.title("Classification Accuracy by Position in Measure")
    # plt.grid(True, axis="y")
    # plt.ylim(0, 1)
    # plt.savefig("manual_accuracy_by_position.png", dpi=300)
    # plt.show()

    return {
        "confusion_matrix": cm,
        "accuracy_by_position": accuracy_by_position,
        "overall_accuracy": sum(
            1 for t, p in zip(all_true_classifications, all_predictions) if t == p
        )
        / len(all_true_classifications),
    }


def manual_decision_tree_main(train_df, test_df, val_df):
    """Main function to test the manual decision tree"""
    # Create an instance of our manual decision tree
    manual_tree = ManualDecisionTree()

    # Print the decision structure
    manual_tree.print_decision_structure()

    # Visualize the decision boundaries
    manual_tree.visualize_tree()

    # Process the test data
    test_pred_df = classify_with_manual_tree(manual_tree, test_df)

    # Evaluate on test set
    test_metrics = evaluate_manual_model(test_df, test_pred_df)
    print(f"Test onset accuracy: {test_metrics['onset_accuracy']:.4f}")
    print(f"Test offset accuracy: {test_metrics['offset_accuracy']:.4f}")

    # Detailed analysis
    analysis = analyze_classification_results_manual(manual_tree, test_df)
    print(f"Overall accuracy: {analysis['overall_accuracy']:.4f}")

    return manual_tree, test_pred_df, analysis


# Example usage:
manual_tree, predictions, analysis = manual_decision_tree_main(train, test, validate)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Import the functions from the previous code
# [All previously defined functions would be here]


def visualize_decision_tree(model, feature_names=None, class_names=None):
    """
    Visualize the decision tree and print out decision rules

    Parameters:
    - model: Trained DecisionTreeClassifier
    - feature_names: Names of features (optional)
    - class_names: Names of classes (optional)
    """
    plt.figure(figsize=(20, 10))
    plot_tree(
        model,
        filled=True,
        feature_names=feature_names,
        class_names=class_names,
        rounded=True,
        fontsize=8,
    )
    plt.title("Decision Tree for MIDI Note Time Classification")
    plt.tight_layout()
    plt.savefig("decision_tree_visualization.png", dpi=300)
    plt.show()

    # Print the decision path rules
    print("Decision Tree Rules:")
    tree_rules = export_text(model, feature_names=feature_names)
    print(tree_rules)


def analyze_decision_boundaries(model):
    """Analyze and visualize the decision boundaries of the tree"""
    # Create a range of times to test (0 to 2 seconds with fine granularity)
    times = np.linspace(0, 2, 1000).reshape(-1, 1)

    # Predict the category for each time
    predictions = model.predict(times)

    # Create a mapping of predicted class to actual sixteenth note time
    predicted_times = [sixteenth_note_times[i] for i in predictions]

    # Plot the mapping from input time to nearest sixteenth note
    plt.figure(figsize=(12, 6))
    plt.plot(times, predicted_times, "b-", linewidth=2)

    # Mark the actual sixteenth note positions
    for time in sixteenth_note_times:
        plt.axhline(y=time, color="r", linestyle="--", alpha=0.3)

    # Add decision boundaries
    for boundary in get_decision_boundaries(model, 0, 2):
        plt.axvline(x=boundary, color="g", linestyle="-", alpha=0.7)

    plt.xlabel("Input Time (seconds)")
    plt.ylabel("Classified Time (seconds)")
    plt.title("Decision Tree Classification Boundaries")
    plt.grid(True)
    plt.savefig("decision_boundaries.png", dpi=300)
    plt.show()


def get_decision_boundaries(model, min_val, max_val, feature_idx=0):
    """Extract the decision boundaries from the tree for a specific feature"""
    boundaries = []

    # Recursive function to extract thresholds from the tree
    def extract_thresholds(tree, node_id=0):
        # If this is a leaf node, return
        if tree.children_left[node_id] == -1:
            return

        # If this node splits on our feature of interest, add the threshold
        if tree.feature[node_id] == feature_idx:
            boundaries.append(tree.threshold[node_id])

        # Continue down both branches
        extract_thresholds(tree, tree.children_left[node_id])
        extract_thresholds(tree, tree.children_right[node_id])

    # Extract the thresholds and filter them to our range of interest
    extract_thresholds(model.tree_)
    valid_boundaries = [b for b in boundaries if min_val <= b <= max_val]
    return sorted(valid_boundaries)


def analyze_classification_results(model, test_df):
    """Analyze how well the model classifies times across the measure"""
    all_norm_times = []
    all_true_classifications = []
    all_predictions = []

    for _, row in test_df.iterrows():
        performance_tuples = (
            eval(row["x_tuples"])
            if isinstance(row["x_tuples"], str)
            else row["x_tuples"]
        )
        performance_duration = row["measure_duration_x"]

        for onset, offset in performance_tuples:
            # Process onset
            norm_onset = normalize_time_to_2sec(onset, performance_duration)
            # Find the ground truth classification (closest sixteenth note)
            true_onset_idx = np.argmin(
                np.abs(np.array(sixteenth_note_times) - norm_onset)
            )
            # Get model prediction
            pred_onset_idx = model.predict([[norm_onset]])[0]

            all_norm_times.append(norm_onset)
            all_true_classifications.append(true_onset_idx)
            all_predictions.append(pred_onset_idx)

            # Process offset (same logic)
            norm_offset = normalize_time_to_2sec(offset, performance_duration)
            true_offset_idx = np.argmin(
                np.abs(np.array(sixteenth_note_times) - norm_offset)
            )
            pred_offset_idx = model.predict([[norm_offset]])[0]

            all_norm_times.append(norm_offset)
            all_true_classifications.append(true_offset_idx)
            all_predictions.append(pred_offset_idx)

    # Create a confusion matrix
    cm = confusion_matrix(all_true_classifications, all_predictions)

    # Plot the confusion matrix
    plt.figure(figsize=(12, 10))
    sns.heatmap(
        cm,
        annot=True,
        fmt="d",
        cmap="Blues",
        xticklabels=[f"{t:.3f}" for t in sixteenth_note_times],
        yticklabels=[f"{t:.3f}" for t in sixteenth_note_times],
    )
    plt.xlabel("Predicted Class")
    plt.ylabel("True Class")
    plt.title("Confusion Matrix for Sixteenth Note Classification")
    plt.tight_layout()
    plt.savefig("confusion_matrix.png", dpi=300)
    plt.show()

    # Analyze accuracy by position in measure
    accuracy_by_position = {}
    for true_idx in range(len(sixteenth_note_times)):
        indices = [i for i, c in enumerate(all_true_classifications) if c == true_idx]
        if indices:
            correct = sum(
                1 for i in indices if all_predictions[i] == all_true_classifications[i]
            )
            accuracy_by_position[sixteenth_note_times[true_idx]] = correct / len(
                indices
            )

    # Plot accuracy by position
    positions = list(accuracy_by_position.keys())
    accuracies = list(accuracy_by_position.values())

    plt.figure(figsize=(12, 6))
    plt.bar(positions, accuracies)
    plt.xlabel("Position in Measure (seconds)")
    plt.ylabel("Classification Accuracy")
    plt.title("Classification Accuracy by Position in Measure")
    plt.grid(True, axis="y")
    plt.ylim(0, 1)
    plt.savefig("accuracy_by_position.png", dpi=300)
    plt.show()

    return {"confusion_matrix": cm, "accuracy_by_position": accuracy_by_position}


def extract_decision_rules(model, feature_names=["time"]):
    """
    Extract human-readable decision rules from the tree
    Returns a list of rules with thresholds and predicted classes
    """
    tree = model.tree_
    feature_name = feature_names[0]
    rules = []

    def recurse(node, depth, parent_threshold=None, direction=""):
        if node == -1:  # Leaf node in tree
            return

        if tree.children_left[node] == -1:  # Leaf node
            class_idx = np.argmax(tree.value[node])
            class_value = sixteenth_note_times[class_idx]

            rule = f"{direction} ({parent_threshold:.3f} sec) → Classify as {class_value:.3f} sec"
            rules.append(rule)
            return

        threshold = tree.threshold[node]

        # Process left child (<=)
        recurse(
            tree.children_left[node], depth + 1, threshold, f"If {feature_name} <= "
        )

        # Process right child (>)
        recurse(
            tree.children_right[node], depth + 1, threshold, f"If {feature_name} > "
        )

    # Start recursion from root
    # Get decision paths for first few levels
    recurse(0, 0)

    return rules


# Enhanced main function
def enhanced_main(train_df, test_df, val_df):
    """Main function with added analysis and visualization"""
    # Original training code
    X_train, y_train = prepare_training_data(train_df)

    # Train model with fixed depth for this example
    final_depth = 5  # You can use the optimal depth found earlier
    model = train_decision_tree(X_train, y_train, max_depth=final_depth)

    # Visualize the tree
    feature_names = ["normalized_time"]
    class_names = [f"{t:.3f}" for t in sixteenth_note_times]
    visualize_decision_tree(model, feature_names, class_names)

    # Analyze decision boundaries
    analyze_decision_boundaries(model)

    # Extract and print human-readable rules
    rules = extract_decision_rules(model, feature_names)
    print("\nKey Decision Rules:")
    for i, rule in enumerate(rules):
        print(f"{i+1}. {rule}")

    # Analyze classification performance
    analysis = analyze_classification_results(model, test_df)

    # Calculate feature importance
    print("\nFeature Importance:")
    print(f"Normalized Time: {model.feature_importances_[0]:.4f}")

    # Additional statistics about the tree
    print("\nDecision Tree Statistics:")
    print(f"Tree Depth: {model.get_depth()}")
    print(f"Number of Leaves: {model.get_n_leaves()}")
    print(f"Number of Nodes: {model.tree_.node_count}")

    # Return the model and analysis results
    return model, analysis


# Example usage:
model, analysis = enhanced_main(train, test, validate)